# Bipolar Junction Transistor (BJT)

## Current-Controlled Amplification

The BJT is a three-terminal device where a small base current controls a larger collector current. It remains important for analog circuits and high-frequency applications.

**Learning Objectives:**
- Understand BJT operation (NPN and PNP)
- Simulate common-emitter characteristics with PADRE
- Generate Gummel plots from simulation
- Extract current gain (beta) from simulated data

In [ ]:
# Setup: Load PADRE environment (required on nanoHUB)
# This cell loads the PADRE simulator into your environment.
# If running locally with PADRE already in your PATH, this will be skipped gracefully.

from nanohubpadre import use

# Load the PADRE simulator environment
%use padre-2.4E-r15

print("PADRE environment setup complete.")

---

## Device Parameters Reference

The `describe()` function shows all available parameters for the BJT factory, including geometry, doping, physical models, and sweep options.

In [ ]:
from nanohubpadre import Simulation

# Show all available parameters for the BJT
Simulation.describe('bjt')

---

## 1. BJT Physics

### 1.1 Structure (NPN)

```
   Emitter (N+)  |  Base (P)  |  Collector (N)
   ============  |  ========  |  =============
   High doping   |  Thin,     |  Low doping
                 |  moderate  |
                 |  doping    |
```

### 1.2 Key Parameters

- **Current gain** ($\beta$ or $h_{FE}$):
  $$\beta = \frac{I_C}{I_B}$$

- **Collector current**:
  $$I_C = I_S \exp\left(\frac{V_{BE}}{V_T}\right)$$

- **Base current**:
  $$I_B = \frac{I_C}{\beta}$$

- **Early voltage** ($V_A$): Models output resistance
  $$I_C = I_S \exp\left(\frac{V_{BE}}{V_T}\right)\left(1 + \frac{V_{CE}}{V_A}\right)$$

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from nanohubpadre import create_bjt

# Physical constants
q = 1.6e-19
kT = 0.0259  # Thermal voltage at 300K

print("BJT Parameters:")
print("="*40)
print(f"Thermal voltage VT = {kT*1000:.1f} mV")
print(f"Typical beta (hFE) = 50-300")
print(f"Typical Early voltage VA = 50-200 V")

---

## 2. Creating and Running a BJT Simulation

Let's create an NPN transistor and run PADRE to examine its characteristics.

In [ ]:
# Create NPN transistor
sim_npn = create_bjt(
    # Geometry
    emitter_width=1.0,      # Emitter region width (um)
    base_width=0.3,         # Base region width (um) - thin!
    collector_width=2.0,    # Collector region width (um)
    device_depth=1.0,
    
    # Mesh
    nx=150,
    ny=30,
    
    # Doping
    emitter_doping=1e20,    # N+ emitter
    base_doping=1e17,       # P base
    collector_doping=1e16,  # N collector
    device_type='npn',
    
    # Models
    temperature=300,
    srh=True,
    auger=True,
    bgn=True,
    
    # Output
    log_bands_eq=True
)

print("NPN BJT Configuration:")
print("="*40)
print("Emitter (N+): 1e20 cm⁻³")
print("Base (P): 1e17 cm⁻³, 0.3 μm wide")
print("Collector (N): 1e16 cm⁻³")

In [ ]:
# Visualize the BJT device structure
sim_npn.device_schematic()

In [ ]:
# View generated deck
print("PADRE Input Deck:")
print("="*60)
print(sim_npn.generate_deck())

In [ ]:
# Run equilibrium simulation
print("Running BJT equilibrium simulation...")
result_eq = sim_npn.run()

if result_eq.returncode == 0:
    print("Simulation completed successfully!")
    print(f"Output directory: {sim_npn.working_dir}")
else:
    print("Simulation failed!")
    print(result_eq.stderr)

In [ ]:
# Plot equilibrium band diagram
print("\nAvailable outputs:")
print(sim_npn.outputs.summary())

In [ ]:
# Plot band diagram
try:
    sim_npn.plot_band_diagram(title="NPN BJT Equilibrium Band Diagram")
except Exception as e:
    print(f"Could not plot band diagram: {e}")

---

## 3. Gummel Plot (Ic, Ib vs Vbe)

The Gummel plot shows collector and base currents vs base-emitter voltage on a semi-log scale. This is the primary characterization tool for BJTs.

In [ ]:
# Gummel plot simulation
sim_gummel = create_bjt(
    emitter_width=1.0,
    base_width=0.3,
    collector_width=2.0,
    device_depth=1.0,
    
    emitter_doping=1e20,
    base_doping=1e17,
    collector_doping=1e16,
    device_type='npn',
    
    temperature=300,
    srh=True,
    auger=True,
    bgn=True,
    
    # Enable I-V logging
    log_iv=True,
    iv_file="gummel",
    
    # Gummel plot: sweep Vbe at fixed Vce
    gummel_sweep=(0.3, 0.85, 0.02),  # Vbe: 0.3 to 0.85V
    gummel_vce=2.0                    # Fixed Vce = 2V
)

print("Gummel Plot Simulation Configuration:")
print("="*40)
print("Vbe sweep: 0.3V to 0.85V")
print("Vce = 2.0V (fixed)")
print(f"Number of bias points: {int((0.85-0.3)/0.02) + 1}")

In [ ]:
# Run the Gummel plot simulation
print("Running Gummel plot simulation...")
print("(This may take a few minutes)")
result_gummel = sim_gummel.run()

if result_gummel.returncode == 0:
    print("Simulation completed successfully!")
else:
    print("Simulation failed!")
    print(result_gummel.stderr)

In [ ]:
# Plot the Gummel plot from simulation
try:
    iv_data = sim_gummel.get_iv_data()
    
    # Get Vbe, Ic, and Ib
    # Electrode assignments: 1=Emitter, 2=Base, 3=Collector (typical)
    Vbe, Ic = iv_data.get_gummel_data(base_electrode=2, collector_electrode=3)
    _, Ib = iv_data.get_gummel_data(base_electrode=2, collector_electrode=2)  # Base current
    
    Vbe = np.array(Vbe)
    Ic = np.abs(np.array(Ic))
    Ib = np.abs(np.array(Ib))
    
    # Create Gummel plot
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=Vbe, y=Ic,
        mode="lines+markers",
        line=dict(color="blue", width=2),
        marker=dict(size=4),
        name="Ic (Collector)"
    ))
    
    fig.add_trace(go.Scatter(
        x=Vbe, y=Ib,
        mode="lines+markers",
        line=dict(color="red", width=2),
        marker=dict(symbol="square", size=4),
        name="Ib (Base)"
    ))
    
    fig.update_yaxes(type="log")
    fig.update_xaxes(title_text="Base-Emitter Voltage Vbe (V)")
    fig.update_yaxes(title_text="Current (A)")
    
    fig.add_annotation(
        text="Slope = q/kT<br>(60 mV/decade)",
        x=0.5, y=1e-9,
        showarrow=False,
        font=dict(size=11)
    )
    
    fig.update_layout(
        title_text="Gummel Plot (PADRE Simulation)",
        template="plotly_white",
        width=1000,
        height=450
    )
    
    fig.show()
    
    print("\nGummel Plot Analysis:")
    print("="*40)
    print(f"Vbe range: {Vbe[0]:.2f}V to {Vbe[-1]:.2f}V")
    print(f"Ic range: {Ic.min():.3e} A to {Ic.max():.3e} A")
    print(f"Ib range: {Ib.min():.3e} A to {Ib.max():.3e} A")
    
except Exception as e:
    print(f"Could not plot Gummel data: {e}")
    print("Trying built-in method...")
    sim_gummel.plot_gummel()

In [ ]:
# Extract current gain (beta) from simulation
try:
    # Calculate beta at each bias point
    mask = (Ic > 1e-12) & (Ib > 1e-12)  # Valid data points
    beta = Ic[mask] / Ib[mask]
    Vbe_valid = Vbe[mask]
    Ic_valid = Ic[mask]
    
    # Plot beta vs Vbe and beta vs Ic
    fig = make_subplots(rows=1, cols=2, subplot_titles=("β vs Vbe", "β vs Ic"))
    
    # Left panel – Beta vs Vbe (linear)
    fig.add_trace(go.Scatter(
        x=Vbe_valid, y=beta,
        mode="lines+markers",
        line=dict(color="green", width=2),
        marker=dict(size=4),
        name="β vs Vbe"
    ), row=1, col=1)
    
    # Right panel – Beta vs Ic (log x)
    fig.add_trace(go.Scatter(
        x=Ic_valid, y=beta,
        mode="lines+markers",
        line=dict(color="green", width=2),
        marker=dict(size=4),
        name="β vs Ic"
    ), row=1, col=2)
    
    fig.update_xaxes(type="log", row=1, col=2)
    
    fig.update_xaxes(title_text="Vbe (V)", row=1, col=1)
    fig.update_yaxes(title_text="Current Gain β", row=1, col=1)
    fig.update_xaxes(title_text="Collector Current Ic (A)", row=1, col=2)
    fig.update_yaxes(title_text="Current Gain β", row=1, col=2)
    
    fig.update_layout(
        template="plotly_white",
        width=1000,
        height=450,
        showlegend=False
    )
    
    fig.show()
    
    # Report statistics
    print("\nCurrent Gain Analysis:")
    print("="*40)
    print(f"Peak β = {beta.max():.1f}")
    print(f"β at Ic=1mA ≈ {beta[np.argmin(np.abs(Ic_valid - 1e-3))]:.1f}" if any(Ic_valid > 1e-3) else "Ic < 1mA")
    print(f"Average β = {beta.mean():.1f}")
    
    # Extract ideality factor
    mask_ideal = (Ic > 1e-10) & (Ic < 1e-5)
    if np.sum(mask_ideal) > 2:
        coeffs = np.polyfit(Vbe[mask_ideal], np.log(Ic[mask_ideal]), 1)
        n_ideality = 1 / (kT * coeffs[0])
        print(f"Ideality factor n ≈ {n_ideality:.2f}")
        
except Exception as e:
    print(f"Could not extract beta: {e}")

---

## 4. Common-Emitter Output Characteristics (Ic vs Vce)

The output characteristic shows collector current vs collector-emitter voltage at different base currents or base voltages.

In [ ]:
# Common-emitter output characteristics
sim_ce = create_bjt(
    emitter_width=1.0,
    base_width=0.3,
    collector_width=2.0,
    device_depth=1.0,
    
    emitter_doping=1e20,
    base_doping=1e17,
    collector_doping=1e16,
    device_type='npn',
    
    temperature=300,
    srh=True,
    bgn=True,
    
    # Enable I-V logging
    log_iv=True,
    iv_file="ic_vce",
    
    # Output characteristic: sweep Vce at fixed Vbe
    vbe=0.7,                      # Base-emitter voltage
    vce_sweep=(0.1, 3.0, 0.1)     # Vce: 0.1 to 3V
)

print("Common-Emitter Output Characteristic Configuration:")
print("="*40)
print("Vbe = 0.7V (fixed)")
print("Vce sweep: 0.1V to 3.0V")

In [ ]:
# Run the output characteristic simulation
print("Running common-emitter output simulation...")
result_ce = sim_ce.run()

if result_ce.returncode == 0:
    print("Simulation completed successfully!")
else:
    print("Simulation failed!")

In [ ]:
# Plot output characteristic
try:
    iv_ce = sim_ce.get_iv_data()
    Vce, Ic_ce = iv_ce.get_output_characteristic(collector_electrode=3)
    
    Vce = np.array(Vce)
    Ic_ce = np.abs(np.array(Ic_ce))
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=Vce, y=Ic_ce * 1e3,
        mode="lines+markers",
        line=dict(color="blue", width=2),
        marker=dict(size=4),
        name="Vbe = 0.7V"
    ))
    
    fig.update_xaxes(title_text="Collector-Emitter Voltage Vce (V)")
    fig.update_yaxes(title_text="Collector Current Ic (mA)")
    
    fig.update_layout(
        title_text="BJT Common-Emitter Output Characteristic (PADRE)",
        template="plotly_white",
        width=1000,
        height=450
    )
    
    fig.show()
    
    # Extract Early voltage from slope
    if len(Vce) > 5:
        # Use linear region (Vce > 0.5V)
        mask = Vce > 0.5
        if np.sum(mask) > 2:
            coeffs = np.polyfit(Vce[mask], Ic_ce[mask], 1)
            slope = coeffs[0]
            Ic_0 = coeffs[1]
            if slope > 0:
                Va = -Ic_0 / slope  # Early voltage
                print(f"\nExtracted Early voltage: Va ≈ {abs(Va):.1f} V")
    
except Exception as e:
    print(f"Could not plot output characteristic: {e}")

In [ ]:
# Simulate output characteristics at multiple Vbe values
Vbe_values = [0.65, 0.70, 0.75, 0.80]
ce_results = {}

print("Simulating output characteristics at multiple Vbe:")
print("="*50)

for Vbe in Vbe_values:
    print(f"\nVbe = {Vbe}V...")
    
    sim = create_bjt(
        emitter_width=1.0,
        base_width=0.3,
        collector_width=2.0,
        emitter_doping=1e20,
        base_doping=1e17,
        collector_doping=1e16,
        device_type='npn',
        temperature=300,
        srh=True,
        log_iv=True,
        iv_file=f"ce_vbe{int(Vbe*100)}",
        vbe=Vbe,
        vce_sweep=(0.1, 3.0, 0.1)
    )
    
    result = sim.run()
    
    if result.returncode == 0:
        try:
            iv = sim.get_iv_data()
            Vce, Ic = iv.get_output_characteristic(collector_electrode=3)
            ce_results[Vbe] = (np.array(Vce), np.abs(np.array(Ic)))
            print(f"  Success")
        except:
            print(f"  Could not parse data")
    else:
        print(f"  Failed")

In [ ]:
# Plot family of output characteristics
if ce_results:
    colors = ['blue', 'green', 'red', 'purple']
    
    fig = go.Figure()
    
    for i, (Vbe, (Vce, Ic)) in enumerate(ce_results.items()):
        color = colors[i % len(colors)]
        fig.add_trace(go.Scatter(
            x=Vce, y=Ic * 1e3,
            mode="lines",
            line=dict(color=color, width=2),
            name=f"Vbe = {Vbe}V"
        ))
    
    fig.update_xaxes(title_text="Collector-Emitter Voltage Vce (V)")
    fig.update_yaxes(title_text="Collector Current Ic (mA)")
    
    fig.add_annotation(
        text="Slopes indicate<br>Early voltage V_A",
        x=2.5, y=0.5,
        showarrow=False,
        font=dict(size=11)
    )
    
    fig.update_layout(
        title_text="BJT Output Characteristics (PADRE Simulation)",
        template="plotly_white",
        width=1000,
        height=450
    )
    
    fig.show()
    
    print("\nOutput characteristic analysis:")
    print("- Active region: Ic relatively constant vs Vce")
    print("- Slope = Ic/VA (Early effect)")
    print("- Higher Vbe → Exponentially higher Ic")

---

## 5. Base Width Effect

The base width is critical for BJT performance. Let's explore its effect using PADRE simulations.

In [ ]:
# Compare different base widths
base_widths = [0.2, 0.3, 0.5]  # microns
width_results = {}

print("Base Width Comparison:")
print("="*50)

for Wb in base_widths:
    print(f"\nWb = {Wb} μm...")
    
    sim = create_bjt(
        emitter_width=1.0,
        base_width=Wb,
        collector_width=2.0,
        emitter_doping=1e20,
        base_doping=1e17,
        collector_doping=1e16,
        device_type='npn',
        temperature=300,
        srh=True,
        log_iv=True,
        iv_file=f"gummel_Wb{int(Wb*10)}",
        gummel_sweep=(0.4, 0.8, 0.02),
        gummel_vce=2.0
    )
    
    result = sim.run()
    
    if result.returncode == 0:
        try:
            iv = sim.get_iv_data()
            Vbe, Ic = iv.get_gummel_data(base_electrode=2, collector_electrode=3)
            _, Ib = iv.get_gummel_data(base_electrode=2, collector_electrode=2)
            
            Vbe = np.array(Vbe)
            Ic = np.abs(np.array(Ic))
            Ib = np.abs(np.array(Ib))
            
            # Calculate beta
            mask = (Ic > 1e-12) & (Ib > 1e-12)
            beta_avg = np.mean(Ic[mask] / Ib[mask]) if np.sum(mask) > 0 else 0
            
            width_results[Wb] = {
                'Vbe': Vbe, 'Ic': Ic, 'Ib': Ib, 'beta': beta_avg
            }
            print(f"  Success, β_avg ≈ {beta_avg:.1f}")
        except:
            print(f"  Could not parse data")
    else:
        print(f"  Failed")

In [ ]:
# Plot base width comparison
if width_results:
    colors = ['blue', 'green', 'red']
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Gummel Plot: Effect of Base Width", "β vs Base Width"))
    
    for i, (Wb, data) in enumerate(width_results.items()):
        color = colors[i % len(colors)]
        label = f'Wb = {Wb} μm (β≈{data["beta"]:.0f})'
        fig.add_trace(go.Scatter(
            x=data['Vbe'], y=data['Ic'],
            mode="lines",
            line=dict(color=color, width=2),
            name=label
        ), row=1, col=1)
    
    fig.update_yaxes(type="log", row=1, col=1)
    fig.update_xaxes(title_text="Vbe (V)", row=1, col=1)
    fig.update_yaxes(title_text="Ic (A)", row=1, col=1)
    
    # Beta vs base width (right panel)
    Wb_list = list(width_results.keys())
    beta_list = [data['beta'] for data in width_results.values()]
    
    fig.add_trace(go.Scatter(
        x=Wb_list, y=beta_list,
        mode="lines+markers",
        line=dict(color="green", width=2),
        marker=dict(size=10),
        name="β vs Wb"
    ), row=1, col=2)
    
    fig.update_xaxes(title_text="Base Width (μm)", row=1, col=2)
    fig.update_yaxes(title_text="Current Gain β", row=1, col=2)
    
    fig.update_layout(
        template="plotly_white",
        width=1000,
        height=450
    )
    
    fig.show()
    
    print("\nKey observation:")
    print("Thinner base → Higher β (less base recombination)")
    print("β ~ 1/Wb for narrow base transistors")

---

## 6. Temperature Effects

BJT characteristics are strongly temperature-dependent.

In [ ]:
# Compare different temperatures
temperatures = [300, 350, 400]  # Kelvin
temp_results = {}

print("Temperature Comparison:")
print("="*50)

for T in temperatures:
    print(f"\nT = {T} K...")
    
    sim = create_bjt(
        emitter_width=1.0,
        base_width=0.3,
        collector_width=2.0,
        emitter_doping=1e20,
        base_doping=1e17,
        collector_doping=1e16,
        device_type='npn',
        temperature=T,
        srh=True,
        log_iv=True,
        iv_file=f"gummel_T{T}",
        gummel_sweep=(0.3, 0.8, 0.02),
        gummel_vce=2.0
    )
    
    result = sim.run()
    
    if result.returncode == 0:
        try:
            iv = sim.get_iv_data()
            Vbe, Ic = iv.get_gummel_data(base_electrode=2, collector_electrode=3)
            temp_results[T] = (np.array(Vbe), np.abs(np.array(Ic)))
            print(f"  Success")
        except:
            print(f"  Could not parse data")
    else:
        print(f"  Failed")

In [ ]:
# Plot temperature comparison
if temp_results:
    colors = ['blue', 'green', 'red']
    
    fig = go.Figure()
    
    for i, (T, (Vbe, Ic)) in enumerate(temp_results.items()):
        color = colors[i % len(colors)]
        kT_val = T * 8.617e-5  # kT in eV
        fig.add_trace(go.Scatter(
            x=Vbe, y=Ic,
            mode="lines",
            line=dict(color=color, width=2),
            name=f"T = {T} K (kT = {kT_val*1000:.1f} mV)"
        ))
    
    fig.update_yaxes(type="log")
    fig.update_xaxes(title_text="Base-Emitter Voltage Vbe (V)")
    fig.update_yaxes(title_text="Collector Current Ic (A)")
    
    fig.update_layout(
        title_text="Temperature Effect on BJT (PADRE Simulation)",
        template="plotly_white",
        width=1000,
        height=450
    )
    
    fig.show()
    
    print("\nTemperature effects:")
    print("- Higher T → Higher Is (saturation current)")
    print("- Slope changes: q/kT decreases with T")
    print("- Vbe for same Ic decreases ~2mV/°C")

---

## 7. 2D Contour Maps

Heat maps of the electrostatic potential, carrier concentrations, and electric field across the BJT cross-section reveal the spatial distribution of the emitter, base, and collector regions and how these change under active-mode bias.

We compare two conditions:
- **Equilibrium** ($V_{be} = V_{ce} = 0$): potential set by doping profiles alone.
- **Active mode** ($V_{be} = 0.7\,\text{V}$, $V_{ce} = 2.0\,\text{V}$): base-emitter junction forward-biased, base-collector junction reverse-biased.

In [ ]:
# Run a simulation that saves Plot3D scatter files at equilibrium and active bias
from nanohubpadre import Solve, Plot3D

sim_2d = create_bjt(
    emitter_width=1.0,
    base_width=0.3,
    collector_width=2.0,
    device_depth=1.0,
    nx=150,
    ny=30,
    emitter_doping=1e20,
    base_doping=1e17,
    collector_doping=1e16,
    device_type='npn',
    temperature=300,
    srh=True,
    auger=True,
    bgn=True,
)

# Equilibrium solve + 3D scatter dumps
sim_2d.add_solve(Solve(initial=True, outfile="eq_2d"))
sim_2d.add_command(Plot3D(potential=True, outfile="pot_eq"))
sim_2d.add_command(Plot3D(doping=True, outfile="dop_eq"))
sim_2d.add_command(Plot3D(electrons=True, outfile="el_eq"))
sim_2d.add_command(Plot3D(holes=True, outfile="hh_eq"))
sim_2d.add_command(Plot3D(e_field=True, outfile="ef_eq"))

# Apply active-mode bias: Vbe = 0.7V (electrode 2), Vce = 2.0V (electrode 3)
sim_2d.add_solve(Solve(project=True, v2=0.7, electrode=2, outfile="vbe_set"))
sim_2d.add_solve(Solve(
    project=True, v3=0.0, vstep=0.2, nsteps=10,
    electrode=3, outfile="bias_2d"
))
sim_2d.add_command(Plot3D(potential=True, outfile="pot_bias"))
sim_2d.add_command(Plot3D(electrons=True, outfile="el_bias"))
sim_2d.add_command(Plot3D(holes=True, outfile="hh_bias"))
sim_2d.add_command(Plot3D(e_field=True, outfile="ef_bias"))
sim_2d.add_command(Plot3D(qfn=True, outfile="qfn_bias"))
sim_2d.add_command(Plot3D(qfp=True, outfile="qfp_bias"))

print("Running simulation for 2D contour maps...")
print("  Equilibrium: pot, dop, el, hh, ef")
print("  Active mode (Vbe=0.7V, Vce=2V): pot, el, hh, ef, qfn, qfp")
result_2d = sim_2d.run()
if result_2d.returncode == 0:
    print("Simulation completed successfully!")
else:
    print("Simulation failed!")
    print(result_2d.stderr)

In [ ]:
# Doping profile (log scale)
sim_2d.plot_contour(
    "dop_eq",
    title="Doping Profile (log₁₀ scale)",
    colorscale="Jet",
    log_scale=True,
    cbar_title="Doping (cm⁻³)",
)

In [ ]:
# Electrostatic potential: equilibrium vs active mode
sim_2d.plot_contour(
    ["pot_eq", "pot_bias"],
    title=["Potential — Equilibrium", "Potential — Vbe=0.7V, Vce=2V"],
    cbar_title="V",
)

In [ ]:
# Electron concentration: equilibrium vs active mode (log scale)
sim_2d.plot_contour(
    ["el_eq", "el_bias"],
    title=["Electrons — Equilibrium", "Electrons — Vbe=0.7V, Vce=2V"],
    colorscale="Hot",
    log_scale=True,
    cbar_title="n (cm⁻³)",
)

In [ ]:
# Hole concentration: equilibrium vs active mode (log scale)
sim_2d.plot_contour(
    ["hh_eq", "hh_bias"],
    title=["Holes — Equilibrium", "Holes — Vbe=0.7V, Vce=2V"],
    colorscale="Hot",
    log_scale=True,
    cbar_title="p (cm⁻³)",
)

In [ ]:
# Quasi-Fermi levels at active bias
sim_2d.plot_contour(
    ["qfn_bias", "qfp_bias"],
    title=["Electron Quasi-Fermi — Active Mode", "Hole Quasi-Fermi — Active Mode"],
    cbar_title="eV",
)

In [ ]:
# Electric field: equilibrium vs active mode
sim_2d.plot_contour(
    ["ef_eq", "ef_bias"],
    title=["Electric Field — Equilibrium", "Electric Field — Vbe=0.7V, Vce=2V"],
    colorscale="Viridis",
    log_scale=True,
    cbar_title="E (V/cm)",
)

---

## 8. Exercises

In [ ]:
# Exercise 1: PNP transistor
sim_pnp = create_bjt(
    emitter_width=1.0,
    base_width=0.3,
    collector_width=2.0,
    emitter_doping=1e20,
    base_doping=1e17,
    collector_doping=1e16,
    device_type='pnp',  # PNP transistor
    temperature=300,
    srh=True,
    log_iv=True,
    iv_file="pnp_gummel",
    gummel_sweep=(-0.3, -0.8, -0.02),  # Negative Vbe for PNP
    gummel_vce=-2.0                     # Negative Vce for PNP
)

print("Running PNP simulation...")
result_pnp = sim_pnp.run()

if result_pnp.returncode == 0:
    print("PNP simulation completed!")
    
    try:
        iv_pnp = sim_pnp.get_iv_data()
        Vbe_p, Ic_p = iv_pnp.get_gummel_data(base_electrode=2, collector_electrode=3)
        
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=np.abs(Vbe_p), y=np.abs(Ic_p),
            mode="lines+markers",
            line=dict(color="red", width=2),
            marker=dict(size=4),
            name="PNP Ic"
        ))
        
        fig.update_yaxes(type="log")
        fig.update_xaxes(title_text="|Vbe| (V)")
        fig.update_yaxes(title_text="|Ic| (A)")
        
        fig.update_layout(
            title_text="PNP BJT Gummel Plot",
            template="plotly_white",
            width=1000,
            height=450
        )
        
        fig.show()
        
        print("\nPNP vs NPN:")
        print("- PNP: P-type emitter and collector, N-type base")
        print("- Holes are majority carriers")
        print("- Generally lower β (hole mobility < electron mobility)")
        print("- Requires negative Vbe and Vce")
        
    except Exception as e:
        print(f"Could not plot: {e}")
else:
    print("Simulation failed")

---

## Summary

In this notebook, you learned:

1. **BJT Structure**: Emitter, Base, Collector regions
2. **Running PADRE Simulations**: Using `create_bjt()` and `sim.run()`
3. **Gummel Plot**: Ic and Ib vs Vbe from PADRE simulation
4. **Output Characteristics**: Ic vs Vce from simulation
5. **Current Gain**: Extracting β from simulated data
6. **Design Parameters**: Base width effect on β
7. **Temperature Effects**: Impact on BJT characteristics
8. **2D Contour Maps**: Visualizing potential, carriers, and fields across the device cross-section

**Key Equations:**
- $I_C = I_S e^{V_{BE}/V_T}$
- $\beta = I_C / I_B$
- Early effect: $I_C = I_S e^{V_{BE}/V_T}(1 + V_{CE}/V_A)$

**Next**: [06 - Solar Cell](06_Solar_Cell.ipynb) - Photovoltaic devices